# Notebook for downloading and processing SST data

## Import all necessary modules

Before running you will need to install podacc subscriber in your terminal

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc4
import os
import requests
import os
from subscriber import podaac_data_downloader as pdd
from subscriber import podaac_access as pa
import argparse
import datetime as dt
import pandas as pd
import statsmodels.api as sm
from scipy.interpolate import interp1d
import datetime as dt

### Download SST for selected dates

In [ ]:
data_path = #The full directory to the file folder you want data to be downloaded to

short_name = 'MUR25-JPL-L4-GLOB-v04.2'
#another option - SMAP SSS: SMAP_JPL_L3_SSS_CAP_MONTHLY_V5
    
start_date_time = #start date for SST download
# must be in format: '2017-06-01T00:00:00Z'
    
end_date_time = #end date for SST download
# must be in format: '2017-06-01T00:00:00Z'

parser = pdd.create_parser()
args = parser.parse_args(['-c',short_name, '-d',data_path,
                              '-sd',start_date_time, '-ed',end_date_time])
    
pdd.run(args)

### Creating SST grids for selected sites

In [25]:
# define a path to your raster files folder 
file_folder = 'D:\MS263\Final_project\SST_data\MUR2002-2023'

# make a file path to one of the nc files
file_path = file_folder + '\\' + '20020902090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc'

# open the file and read in the SST variable to a 2D numpy grid
ds = nc4.Dataset(file_path)
sst_grid = ds.variables['analysed_sst']
sst_grid = np.array(sst_grid)
ds.close()

# print the minimum and maximum of the grid
print(np.max(sst_grid))
print(np.min(sst_grid))

307.44899999999996
-32768.0


In [1]:
#create grid for a site by inputting min & max coordinates

min_x_n_OR = -124.663775
max_x_n_OR = -124.390232
min_y_n_OR = 44.607492
max_y_n_OR = 45.535849

ds = nc4.Dataset(file_path)
x_grid_n_OR = ds.variables['lon']
x_grid_n_OR = np.array(x_grid_n_OR)
y_grid_n_OR = ds.variables['lat']
y_grid_n_OR = np.array(y_grid_n_OR)
sst_grid_n_OR = ds.variables['analysed_sst']
sst_grid_n_OR = np.array(sst_grid_n_OR)
ds.close()

min_x_index_n_OR = np.argmin(np.abs(min_x_n_OR - x_grid_n_OR))
max_x_index_n_OR = np.argmin(np.abs(max_x_n_OR - x_grid_n_OR))

min_y_index_n_OR = np.argmin(np.abs(min_y_n_OR - y_grid_n_OR))
max_y_index_n_OR = np.argmin(np.abs(max_y_n_OR - y_grid_n_OR))

print(min_x_index_n_OR)
print(min_y_index_n_OR)
print(max_x_index_n_OR)
print(max_y_index_n_OR)

NameError: name 'nc4' is not defined

### Calculate average winter SST values for your site for every year

In [39]:
# make a sorted file list of all the SST nc files
file_list = []
for file_name in os.listdir(file_folder):
    if file_name[-3:]=='.nc':
        file_list.append(file_name)

# sort the file list
file_list.sort()

# make empty lists called year_list_site and sst_values_site
# to store data from all of the files
print(file_list[0])
print(file_list[-1])

year_list_n_OR = []
sst_values_n_OR = []


for yr in range(2002, 2023):
    print('working on year' + str(yr))
    sum=0
    count=0
    min_date = dt.datetime(yr, 12, 1)
    max_date = dt.datetime(yr+1, 2, 28)
# loop through all of the files in the file list
    for file_name in file_list:
        
        
        # make a date string for the file
        file_parts = file_name.split('-')
        year = file_parts[0][0:4]
        month = file_parts[0][4:6]
        day = file_parts[0][6:8]
        date_str = year+'-'+month+'-'+day

        file_date = dt.datetime(int(year),int(month),int(day))

        if file_date >= min_date and file_date < max_date:
            
        
            # open the file to get the SST grid
            file_path = file_folder + '\\' + file_name
            ds = nc4.Dataset(file_path)
            longitude = np.array(ds.variables['lon'])
            latitude = np.array(ds.variables['lat'])
            sst = np.array(ds.variables['analysed_sst'])
            ds.close()
            sst = sst[0,:,:]
        
        
            sst_grid_subset = sst[min_y_index_n_OR:max_y_index_n_OR, min_x_index_n_OR:max_x_index_n_OR]
            
            sst_grid_subset = sst_grid_subset[sst_grid_subset>-100]
            

            sum=sum+np.mean(sst_grid_subset)
            count = count +1

    average = sum/count
    sst_values_n_OR.append(average)
    year_list_n_OR.append(yr+1)
            

print(year_list_n_OR)
print(sst_values_n_OR)

20020901090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc
20230929090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc
working on year2002
working on year2003
working on year2004
working on year2005
working on year2006
working on year2007
working on year2008
working on year2009
working on year2010
working on year2011
working on year2012
working on year2013
working on year2014
working on year2015
working on year2016
working on year2017
working on year2018
working on year2019
working on year2020
working on year2021
working on year2022
[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
[284.2275758426967, 283.54558707865164, 283.382547752809, 283.5626123595505, 283.16316011235955, 282.1521910112359, 282.5767556179774, 283.4795280898876, 283.0910252808988, 282.7450393258426, 283.31749999999994, 282.7239157303371, 285.8072106741572, 284.4582668539325, 284.02996910112364, 283.8183230337079, 284.1218781249999, 

### Convert from Kelvin to Celsius

In [40]:
sst_values_n_OR_c = []

for year in sst_values_n_OR:
    sst_values_n_OR_c.append(year-273.15)
    print(year-273.15)

print(sst_values_n_OR_c)

11.077575842696717
10.395587078651658
10.232547752809012
10.412612359550508
10.013160112359571
9.002191011235936
9.426755617977449
10.329528089887617
9.941025280898828
9.595039325842606
10.167499999999961
9.573915730337148
12.657210674157227
11.308266853932537
10.879969101123663
10.668323033707907
10.971878124999932
10.008699367088752
10.291948051948111
10.054706249999867
9.854712500000062
[11.077575842696717, 10.395587078651658, 10.232547752809012, 10.412612359550508, 10.013160112359571, 9.002191011235936, 9.426755617977449, 10.329528089887617, 9.941025280898828, 9.595039325842606, 10.167499999999961, 9.573915730337148, 12.657210674157227, 11.308266853932537, 10.879969101123663, 10.668323033707907, 10.971878124999932, 10.008699367088752, 10.291948051948111, 10.054706249999867, 9.854712500000062]


### Export as a CSV

In [41]:
sst_timeseries_interpolated = np.column_stack([year_list_n_OR, sst_values_n_OR_c])

# define an output folder
output_folder = 'C:\\Users\\maddi\\Desktop\\MS263\\MS263_FinalProject\\data\\Site_SST'

# output the interpolated wind file
np.savetxt(output_folder+'\\sst_n_oregon_final_timeseries.csv', sst_timeseries_interpolated, delimiter=',')